## Data curation and dimensionalization

Through this process we create our fact and dimension tables to build out our initial curated model.

#### Date Dimension
Using a combination of PySpark and SparkSQL we create our date dimension. This combination of code using magic commands also for a large date dimension table to be created quickly.

In [1]:
%%pyspark
from pyspark.sql.functions import explode, sequence, to_date

beginDate = '2000-01-01'
endDate = '2050-12-31'

spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as calendarDate").createOrReplaceTempView('dates')

StatementMeta(, f14f68b4-133c-4c11-8166-018c208839b8, 3, Finished, Available)

In [2]:
create or replace table DimDate 
USING delta
as
select
  year(calendarDate) * 10000 + month(calendarDate) * 100 + day(calendarDate) as dateInt,
  CalendarDate,
  year(calendarDate) AS CalendarYear,
  date_format(calendarDate, 'MMMM') as CalendarMonth,
  month(calendarDate) as MonthOfYear,
  date_format(calendarDate, 'EEEE') as CalendarDay,
  dayofweek(calendarDate) AS DayOfWeek,
  weekday(calendarDate) + 1 as DayOfWeekStartMonday,
  case
    when weekday(calendarDate) < 5 then 'Y' else 'N' end as IsWeekDay, dayofmonth(calendarDate) as DayOfMonth, case when calendarDate = last_day(calendarDate) then 'Y' else 'N' end as IsLastDayOfMonth, dayofyear(calendarDate) as DayOfYear, weekofyear(calendarDate) as WeekOfYearIso, quarter(calendarDate) as QuarterOfYear, /* Use fiscal periods needed by organization fiscal calendar */ case when month(calendarDate) >= 10 then year(calendarDate) + 1
    else year(calendarDate)
  end as FiscalYearOctToSep,
  (month(calendarDate) + 2) % 12 + 1 AS FiscalMonthOctToSep,
  case
    when month(calendarDate) >= 7 then year(calendarDate) + 1
    else year(calendarDate)
  end as FiscalYearJulToJun,
  (month(calendarDate) + 5) % 12 + 1 AS FiscalMonthJulToJun
from
  dates
order by
  calendarDate

StatementMeta(, f14f68b4-133c-4c11-8166-018c208839b8, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

#### User Dimension
Here we create our user dimension by combining our User table, their current default address, and current subscription status.

In [3]:
DROP TABLE IF EXISTS DimUser;

CREATE TABLE DimUser AS
SELECT u.Id as UserId, u.FirstName, u.LastName, u.Email, u.PhoneNumber, MemberSince,
    ua.Address, ua.State, ua.ZipCode, us.RenewalDay, us.Active as IsSubscriptionActive
        FROM appusers as u JOIN appuseraddresses as ua ON u.Id = ua.UserId
        JOIN appusersubscriptionstatus as us ON u.Id = us.UserId
    WHERE ua.Default = true

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### Kiosk Dimension
Quick load of our Kiosk table directly with no major changes.

In [4]:
DROP TABLE IF EXISTS DimKiosk;
CREATE TABLE DimKiosk AS
SELECT Id as KioskId, Address, State, ZipCode, InstallDate FROM appkiosk

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### Movie Dimension
Here we load our movie dimension, the only join is to turn the Genre into natural language.

In [5]:
DROP TABLE IF EXISTS DimMovies;
CREATE TABLE DimMovies AS
SELECT m.movie_id as MovieId, m.title as Title, m.mpaa_rating as MpaaRating, g.genre as Genre, m.poster_url as PosterImageUrl, m.release_date as ReleaseDate
from dbomovies as m
JOIN dbogenres as g ON m.genre_id = g.genre_id

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### Purchases Fact
Here we take all our purchase line items and load them as our purchase facts. We get the purchasing user from the applications join table, and we turn the product/inventory item into natural language instead of an id. We also join our date dimension for use in reporting.

In [6]:
DROP TABLE IF EXISTS FactPurchases;
CREATE TABLE FactPurchases AS
SELECT pli.Id as PurchaseLineItemId, p.Id as PurchaseId, pj.PurchasingUsersId, p.PurchaseLocationId, d.dateInt as TransactionCreatedOnDateId, 
        pli.Quantity, pli.TotalPrice, i.ItemDescription 
    FROM apppurchases as p
    JOIN apppurchaselineitems as pli ON p.Id = pli.PurchaseId
    JOIN apppurchaseuser as pj ON p.Id = pj.PurchasesId
    JOIN appinventory as i ON pli.ItemId = i.Id
    JOIN dimdate as d ON CAST(p.TransactionCreatedOn as date) = d.CalendarDate

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### Rentals Fact
Here we combine our Rentals data, with our purchases and returns, we also join our Date Dimension. The goal of this fact is to get all data in a single row about a rental so that our reports can be as efficient as possible.

In [7]:
DROP TABLE IF EXISTS FactRentals;
CREATE TABLE FactRentals AS
SELECT r.Id as RentalId, r.MovieId, r.UserId, pur.PurchaseLocationId as RentalLocationId, d.dateInt as RentalDateId, dr.dateInt as ExpectedReturnDateId, drt.dateInt as ReturnDateId, purt.PurchaseLocationId as ReturnLocationId, rt.LateDays, 
    pr.TotalPrice as RentalPrice, prt.TotalPrice as LateFee, (RentalPrice + COALESCE(LateFee, 0)) as TotalPrice, current_timestamp as DateModified
    FROM apprentals as r
    JOIN apppurchaselineitems as pr ON r.PurchaseLineItemId = pr.Id
    JOIN apppurchases as pur ON pr.PurchaseId = pur.Id
    JOIN dimdate as d ON CAST(r.RentalDate as date) = d.CalendarDate
    JOIN dimdate as dr on CAST(r.ExpectedReturnDate as date) = dr.CalendarDate
    LEFT JOIN appreturns as rt ON r.Id = rt.RentalId
    LEFT JOIN apppurchaselineitems as prt ON rt.LateChargeLineItemId = prt.Id
    LEFT JOIN apppurchases as purt ON prt.PurchaseId = purt.Id
    LEFT JOIN dimdate as drt ON drt.CalendarDate = CAST(rt.ReturnDate as date)


StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>